0813更新：

combine-model： 把词向量和字向量信息全部丢进来训练。

一共有4路输入，wd 和 ch 各两路。

与 c1-1 中不同：
- 窗口长度变长，由[2,3,4,5,7]->[3,4,5,7,10]
- 拼接方式，原来是对 title 和 content 输出求max，现在是直接 concat
- 初始化方式，原来使用 Xavier 初始化，现在使用 normal 初始化

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import os
import sys
import shutil
import time


time0 = time.time()
print('Starting ...')
model_name = 'c1-2-cnn-256-345710'                    # 模型名称
wd_embedding = np.load('../data/W_embedding.npy').astype(np.float32)                      # 导入预训练好的词向量
ch_embedding = np.load('/home1/huangyongye/mygithub/zhihu/data/ch_W_embedding.npy').astype(np.float32)  # 字向量
model_path = '../ckpt/' + model_name + '/'                  # 模型保存位置
summary_path = '../summary/' + model_name + '/'             # summary 位置
result_path = '../result/' + model_name + '.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '.npy'            # scores.npy 位置
local_scores_path = '../local_scores/' + model_name + '.npy'


if not os.path.exists(model_path):
    os.makedirs(model_path)         
model_path = model_path + 'model.ckpt'
if os.path.exists(summary_path):   # 删除原来的 summary 文件，避免重合
    print('removed the existing summary files.')
    shutil.rmtree(summary_path)
os.makedirs(summary_path)          # 然后再次创建
    
# ##################### config ######################
n_step1 = wd_title_len = 50                   # title句子长度
n_step2 = wd_content_len = 150                # content 长度
n_step3 = ch_title_len = 52
n_step4 = ch_content_len = 300
input_size = embedding_size = 256       # 字向量长度
n_class = 1999                          # 类别总数
filter_sizes = [3,4,5,7,10]                  # 卷积核大小
n_filter = 256                          # 每种卷积核的个数
fc_hidden_size = 1024                   # fc 层节点数
n_filter_total = n_filter * len(filter_sizes)
print('Prepared, costed time %g s.' % (time.time() - time0))

Starting ...
removed the existing summary files.
Prepared, costed time 9.25318 s.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers

'''
双端 GRU，知乎问题多标签分类。
'''
print('Building model ...')
keep_prob = tf.placeholder(tf.float32, [])
lr = tf.placeholder(tf.float32)
batch_size = tf.placeholder(tf.int32, [])  # 注意类型必须为 tf.int32
tst = tf.placeholder(tf.bool)
n_updates = tf.placeholder(tf.int32)      # training iteration,传入 bn 层
update_emas = list()                       # BN 层中所有的更新操作

def weight_variable(shape, name):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.get_variable(name=name, initializer=initial, dtype=tf.float32)

def bias_variable(shape, name):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.get_variable(name=name, initializer=initial, dtype=tf.float32)

def batchnorm(Ylogits, is_test, num_updates, offset, convolutional=False):
    """batchnormalization.
    Args:
        Ylogits: 1D向量或者是3D的卷积结果。
        num_updates: 迭代的global_step
        offset：表示beta，全局均值；在 RELU 激活中一般初始化为 0.1。
        scale：表示lambda，全局方差；在 sigmoid 激活中需要，这 RELU 激活中作用不大。
        m: 表示batch均值；v:表示batch方差。
        bnepsilon：一个很小的浮点数，防止除以 0.
    Returns:
        Ybn: 和 Ylogits 的维度一样，就是经过 Batch Normalization 处理的结果。
        update_moving_everages：更新mean和variance，主要是给最后的 test 使用。
    """
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, num_updates) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages

with tf.device('/gpu:0'):
    with tf.variable_scope('embedding') as vs:
        word_embedding = tf.get_variable(name="word_embedding", shape=wd_embedding.shape, 
                            initializer=tf.constant_initializer(wd_embedding), trainable=True)   # fine-tune
        char_embedding = tf.get_variable(name="char_embedding", shape=ch_embedding.shape, 
                            initializer=tf.constant_initializer(ch_embedding), trainable=True)   # fine-tune
        
with tf.name_scope('Inputs'):
    X1_inputs = tf.placeholder(tf.int64, [None, n_step1], name='X1_input')  # wd_title
    X2_inputs = tf.placeholder(tf.int64, [None, n_step2], name='X2_input')  # wd_content
    X3_inputs = tf.placeholder(tf.int64, [None, n_step3], name='X3_input')  # ch_title
    X4_inputs = tf.placeholder(tf.int64, [None, n_step4], name='X4_input')  # ch_content
    y_inputs = tf.placeholder(tf.float32, [None, n_class], name='y_input')    
    inputs1 = tf.nn.embedding_lookup(word_embedding, X1_inputs)
    inputs2 = tf.nn.embedding_lookup(word_embedding, X2_inputs)
    inputs3 = tf.nn.embedding_lookup(char_embedding, X3_inputs)
    inputs4 = tf.nn.embedding_lookup(char_embedding, X4_inputs)


def textcnn(inputs, n_step):
    """build the TextCNN network. Return the h_drop"""
    # inputs.shape = [batchsize, n_step, embedding_size]
    inputs = tf.expand_dims(inputs, -1)
    pooled_outputs = list()
    for i, filter_size in enumerate(filter_sizes):
        with tf.variable_scope("conv-maxpool-%s" % filter_size):
            # Convolution Layer
            filter_shape = [filter_size, embedding_size, 1, n_filter]
            W_filter = weight_variable(shape=filter_shape, name='W_filter')
            beta = bias_variable(shape=[n_filter], name='beta_filter')
            tf.summary.histogram('beta_filter', beta)
            conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
            conv_bn, update_ema = batchnorm(conv, tst, n_updates, beta, convolutional=True)    # 在激活层前面加 BN
            # Apply nonlinearity, batch norm scaling is not useful with relus
            # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
            h = tf.nn.relu(conv_bn, name="filter_relu")
            # Maxpooling over the outputs
            pooled = tf.nn.max_pool(h,ksize=[1, n_step - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],padding='VALID',name="pool")
            pooled_outputs.append(pooled)
            update_emas.append(update_ema)
    h_pool = tf.concat(pooled_outputs, 3)
    h_pool_flat = tf.reshape(h_pool, [-1, n_filter_total]) 
    return h_pool_flat    # shape = [-1, n_filter_total]


with tf.variable_scope('wd-encoder'):    
    with tf.variable_scope('wd-title'):
        wd_output_title = textcnn(inputs1, n_step1)  # shape=[batch_size, n_filter_total]
    with tf.variable_scope('wd-content'):
        wd_output_content = textcnn(inputs2, n_step2)
    
with tf.variable_scope('ch-encoder'):
    with tf.variable_scope('ch-title'):
        ch_output_title = textcnn(inputs3, n_step3)
    with tf.variable_scope('ch-content'):
        ch_output_content = textcnn(inputs4, n_step4)  # shape=[batch_size, n_filter_total]
    
    
with tf.variable_scope('fc-bn-layer'):
    output = tf.concat([wd_output_title, wd_output_content, ch_output_title, ch_output_content], axis=1)  # shape=[batch_size, n_filter_total*4]
    W_fc = weight_variable([n_filter_total*4, fc_hidden_size], name='Weight_fc')
    tf.summary.histogram('W_fc', W_fc)
    h_fc = tf.matmul(output, W_fc, name='h_fc')
    beta_fc = bias_variable([fc_hidden_size], name="beta_fc")
    tf.summary.histogram('beta_fc', beta_fc)
    fc_bn, update_ema_fc = batchnorm(h_fc, tst, n_updates, beta_fc, convolutional=False)
    update_emas.append(update_ema_fc)
    fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
    fc_bn_drop = tf.nn.dropout(fc_bn_relu, keep_prob, name="fc_dropout")

with tf.variable_scope('out_layer'):
    W_out = weight_variable(shape=[fc_hidden_size, n_class], name='Weight_out')
    tf.summary.histogram('Weight_out', W_out)
    b_out = bias_variable([n_class], name='bias_out') 
    tf.summary.histogram('bias_out', b_out)
    y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  #每个类别的分数 scores
    
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_inputs))
    tf.summary.scalar('cost', cost)

#  ------------- 优化器设置 ---------------------
with tf.name_scope('AdamOptimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    train_op = optimizer.minimize(cost)
    update_op = tf.group(*update_emas)   # 更新 BN 参数

# summary
merged = tf.summary.merge_all() # summary
train_writer = tf.summary.FileWriter(summary_path + 'train', sess.graph)
test_writer = tf.summary.FileWriter(summary_path + 'test')
print 'Finished creating the TextCNN model.'

Building model ...
Finished creating the TextCNN model.


## 导入数据

In [3]:
sys.path.append('..')
from data_helpers import BatchGenerator
from data_helpers import to_categorical
from evaluator import score_eval


wd_train_path = '/home/huangyongye/zhihu_data/data_train/'
wd_valid_path = '/home/huangyongye/zhihu_data/data_valid/'
ch_train_path = '../ch-data/data_train/'
ch_valid_path = '../ch-data/data_valid/'


tr_batches = os.listdir(wd_train_path)   # batch 文件名列表
va_batches = os.listdir(wd_valid_path)
n_tr_batches = len(tr_batches)
n_va_batches = len(va_batches)

# 测试用
# n_tr_batches = 500  
# n_va_batches = 50
print('n_tr_batch=%d' % n_tr_batches)
print('n_va_batches=%d' % n_va_batches)


def get_batch(batch_id, mode='train'):
    """get a batch from data_path.
    model: 'train', 'valid' 三种方式。
    """
    if mode == 'train':
        wd_path = wd_train_path
        ch_path = ch_train_path 
    if mode == 'valid':
        wd_path = wd_valid_path
        ch_path = ch_valid_path
    wd_batch = np.load(wd_path + str(batch_id) + '.npz')
    wd_X_batch = wd_batch['X']
    X1_batch = wd_X_batch[:, :n_step1]
    X2_batch = wd_X_batch[:, n_step1:]
    ch_batch = np.load(ch_path + str(batch_id) + '.npz')
    ch_X_batch = ch_batch['X']
    X3_batch = ch_X_batch[:, :n_step3]
    X4_batch = ch_X_batch[:, n_step3:]
    y_batch = ch_batch['y']
    return [X1_batch, X2_batch, X3_batch, X4_batch, y_batch]


def valid_epoch():
    """Test on the valid data."""
    _costs = 0.0
    predict_labels_list = list()  # 所有的预测结果
    marked_labels_list = list()   # 真实标签
    for i in xrange(n_va_batches):
        [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(i, mode='valid')
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, y_pred]  
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, y_inputs:y_batch, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        summary, _cost, predict_labels = sess.run(fetches, feed_dict)
        _costs += _cost
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    mean_cost = _costs / n_va_batches
    return mean_cost, precision, recall, f1

print('Every thing prepared!')

n_tr_batch=22656
n_va_batches=782
Every thing prepared!


##  模型训练

之前的训练速度 9334.15 s/epoch

In [8]:
8e-4 * (0.75**11)

3.3788108825683595e-05

In [4]:
_lr = 8e-4 
decay = 0.75 
# 正式
max_epoch = 3           # 中间保存，以防断电
max_max_epoch = 9       # 最多迭代的次数
decay_step = 15000
valid_step = 8000        # 每 valid_step 就进行一次 valid 运算

# 测试
# max_epoch = 1            # 超多 max_epoch 以后开始降低 lr
# max_max_epoch = 3       # 最多迭代的次数
# decay_step = 500
# valid_step = 250

print('tr_batch_num=%d' % n_tr_batches)
saver = tf.train.Saver(max_to_keep=3)           # 最多保存的模型数量
sess.run(tf.global_variables_initializer())
last_f1 = 0.40
model_num = 0
global_step = 0
print('OK!')

tr_batch_num=22656
OK!


In [5]:
time0 = time.time()

for epoch in xrange(max_max_epoch):
    batch_indexs = np.random.permutation(n_tr_batches)  # shuffle the training data
    if epoch == max_epoch:   # 中间保存一次，一方断电
        if model_num == 0:
            model_num += 1
            save_path = saver.save(sess, model_path, global_step=model_num)
            print('First Saver! The save path is ', save_path)
    print 'EPOCH %d, lr= %g' % (epoch+1, _lr)    
    for batch in xrange(n_tr_batches): 
        global_step += 1
        if (global_step+1) % decay_step == 0:    # 降低学习率
            _lr = _lr * decay
        if (global_step+1) % valid_step == 0:    # 进行 valid 计算
            valid_cost, precision, recall, f1 = valid_epoch()
            print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g, time=%g s' % (
                    global_step+1, valid_cost, precision, recall, f1, time.time()-time0))
            time0 = time.time()
            if (f1 > last_f1):
                last_f1 = f1
                model_num += 1
                save_path = saver.save(sess, model_path, global_step=model_num)
                print('the save path is ', save_path) 
                
        batch_id = batch_indexs[batch]
        [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(batch_id, mode='train')
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, train_op, update_op]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, y_inputs:y_batch, 
                     batch_size:_batch_size, lr:_lr, keep_prob:0.5, tst:False, n_updates:global_step}
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        if global_step % 500:
            train_writer.add_summary(summary, global_step)
            batch_id = np.random.randint(0, n_va_batches)   # 随机选一个验证batch
            [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(batch_id, mode='valid')
            y_batch = to_categorical(y_batch)
            _batch_size = len(y_batch)
            feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, y_inputs:y_batch, 
                         batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)

valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('Global_step=%d;  valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
    global_step+1, valid_cost, precision, recall, f1, time.time()-time0) )
if (f1 > last_f1):
    model_num += 1
    save_path = saver.save(sess, model_path, global_step=model_num)
    print('the save path is ', save_path) 

EPOCH 1, lr= 0.0008
Global_step=8000: valid cost=0.0105595; p=0.96138, r=0.404641, f1=0.284779, time=7950.93 s
Global_step=16000: valid cost=0.00772768; p=1.14878, r=0.477489, f1=0.337293, time=6911.4 s
EPOCH 2, lr= 0.0006
Global_step=24000: valid cost=0.00671484; p=1.23948, r=0.513451, f1=0.363056, time=6940.13 s
Global_step=32000: valid cost=0.00557501; p=1.30346, r=0.536398, f1=0.380015, time=6875.14 s
Global_step=40000: valid cost=0.00608487; p=1.30528, r=0.538036, f1=0.380992, time=6892.26 s
EPOCH 3, lr= 0.0003375
Global_step=48000: valid cost=0.00483268; p=1.36604, r=0.559748, f1=0.397052, time=6892.77 s
Global_step=56000: valid cost=0.00518072; p=1.35263, r=0.554617, f1=0.393337, time=6872.65 s
Global_step=64000: valid cost=0.00475497; p=1.3878, r=0.567632, f1=0.402857, time=6878.73 s
('the save path is ', '../ckpt/c1-2-cnn-256-345710/model.ckpt-1')
EPOCH 4, lr= 0.000253125
Global_step=72000: valid cost=0.00457005; p=1.39101, r=0.569128, f1=0.403881, time=7491.22 s
('the save pa

KeyboardInterrupt: 

- Bi-LSTM 模型<br/>
**batch_size=256，迭代12个epoch，基本收敛。结果： 验证集 f1=0.38618； 提交结果 0.3873186**
- Bi-GRU 模型<br/>
**batch_size=256，迭代15个epoch。大概在 13 个 epoch 就收敛了。结果： 验证集 f1=0.390534； 提交结果 0.39198**
- attention-Bi-GRU 模型<br/>
**batch_size=256，迭代18个epoch。在16个epoch收敛。结果：验证集 f1=f1=0.391734 ；提交结果 0.39310**


- textcnn-256 lr=1e-3, decay=0.65, dropout=0.5, 迭代6次基本收敛， f1=0.388
- textcnn-256-bn lr=1.5E-3，decay=0.65, dropout=0.5, 6次基本收敛，8次好一点， f1=0.389
- textcnn-fc-drop 迭代 4.5 个epoch 收敛。

## 本地测试
使用 seed13 的前 10万条数据.

In [6]:
# 导入保存好的模型
# saver = tf.train.Saver()
# best_model_path = model_path + '-' + str(8)  # 导入最优模型
# saver.restore(sess, best_model_path)
# print('Finished loading model.')

from tqdm import tqdm
local_scores_path = '../local_scores/' + model_name + '.npy'

# 导入测试数据
def local_predict(scores_path=local_scores_path):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    time0 = time.time()
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    marked_labels_list = list()   # 真实标签
    for i in tqdm(xrange(n_va_batches)):
        [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(i, mode='valid')
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, y_inputs:y_batch, 
                         batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    print('local valid p=%g, r=%g, f1=%g; speed=%g s/epoch' % ( precision, recall, f1, time.time()-time0) )
    np.save(local_scores_path, predict_scores)
    print('Writed the scores into %s, time %g s' % (local_scores_path, time.time()-time0))
    
local_predict()

  0%|          | 0/782 [00:00<?, ?it/s]

local predicting ...


100%|██████████| 782/782 [01:28<00:00,  8.88it/s]


local valid p=1.41705, r=0.57794, f1=0.410514; speed=88.542 s/epoch
Writed the scores into ../local_scores/c1-2-cnn-256-345710.npy, time 89.199 s


In [8]:
model_num += 1
save_path = saver.save(sess, model_path, global_step=model_num)
print('the save path is ', save_path) 

('the save path is ', '../ckpt/c1-2-cnn-256-345710/model.ckpt-7')


#### 最后3 个epoch 取平均值

In [14]:
# 导入保存好的模型
# saver = tf.train.Saver()
# best_model_path = model_path + '-' + str(8)  # 导入最优模型
# saver.restore(sess, best_model_path)
# print('Finished loading model.')

from tqdm import tqdm

marked_labels_list = np.load('../data/marked_labels_list.npy')
# # 求 softmax
def _softmax(score):
    """对一个样本的输出类别概率进行 softmax 归一化.
    score: arr.shape=[1999].
    """
    max_sc = np.max(score)   # 最大分数
    score = score - max_sc
    exp_sc = np.exp(score)
    sum_exp_sc = np.sum(exp_sc)
    softmax_sc = exp_sc / sum_exp_sc
    return softmax_sc    # 归一化的结果
    
def softmax(scores):
    """对所有样本的输出概率进行 softmax 归一化处理。
    scores: arr.shape=[n_sample, 1999].
    """
    softmax_scs = map(_softmax, scores)
    return np.asarray(softmax_scs)


# 导入测试数据
def local_predict(epoch, scores_path=local_scores_path):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    saver = tf.train.Saver()
    best_model_path = model_path + '-' + str(epoch)  # 导入最优模型
    saver.restore(sess, best_model_path)
    print('Finished loading model.')

    time0 = time.time()
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    marked_labels_list = list()   # 真实标签
    for i in tqdm(xrange(n_va_batches)):
        [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(i, mode='valid')
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, y_inputs:y_batch, 
                         batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    print('local valid p=%g, r=%g, f1=%g; speed=%g s/epoch' % ( precision, recall, f1, time.time()-time0) )
#     np.save(local_scores_path, predict_scores)
#     print('Writed the scores into %s, time %g s' % (local_scores_path, time.time()-time0))
    return softmax(predict_scores)   # 返回结果
    
local_scores_path = '../local_scores/' + model_name + '-all.npy'
sum_scores = np.zeros((len(marked_labels_list), 1999), dtype=float)   
last_f1 = 0.410514
valid_epoch = [5,6,7]
for epoch in valid_epoch:
    score =  np.vstack(local_predict(epoch))
    sum_scores = sum_scores + score
predict_labels_list = map(lambda label: label.argsort()[-1:-6:-1], sum_scores) # 取最大的5个下标
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print('p=%g\tr=%g\tf1=%g;' % ( precision, recall, f1))
if f1 > last_f1:
    np.save(local_scores_path, sum_scores)
    print('save scores to %s' % local_scores_path)


local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/c1-2-cnn-256-345710/model.ckpt-5


  0%|          | 1/782 [00:00<01:28,  8.81it/s]

Finished loading model.


100%|██████████| 782/782 [01:27<00:00,  8.97it/s]


local valid p=1.41388, r=0.577423, f1=0.409986; speed=87.6557 s/epoch
local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/c1-2-cnn-256-345710/model.ckpt-6


...

local valid p=1.41598, r=0.577833, f1=0.41037; speed=88.532 s/epoch
local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/c1-2-cnn-256-345710/model.ckpt-7


  0%|          | 1/782 [00:00<01:19,  9.81it/s]

Finished loading model.


100%|██████████| 782/782 [01:26<00:00,  8.99it/s]


local valid p=1.41705, r=0.57794, f1=0.410514; speed=87.5181 s/epoch
p=1.42491	r=0.581523	f1=0.412981;
save scores to ../local_scores/c1-2-cnn-256-345710-all.npy


## 对测试数据进行预测

In [5]:
def get_test_batch(batch_id):
    """get a batch from data_path.
    model: 'train', 'valid' 三种方式。
    """
    wd_path = '../wd-data/data_test/'
    ch_path = '../ch-data/data_test/'
    wd_X_batch = np.load(wd_path + str(batch_id) + '.npy')
    X1_batch = wd_X_batch[:, :n_step1]
    X2_batch = wd_X_batch[:, n_step1:]
    ch_X_batch = np.load(ch_path + str(batch_id) + '.npy')
    X3_batch = ch_X_batch[:, :n_step3]
    X4_batch = ch_X_batch[:, n_step3:]
    return [X1_batch, X2_batch, X3_batch, X4_batch]


In [6]:
from tqdm import tqdm
n_te_batches = len(os.listdir('../ch-data/data_test/'))
global_step = 100000

# 导入测试数据
def predict(epoch):
    """预测测试集结果，并保存到  result.csv 文件中。"""
    print('local predicting ...')
    saver = tf.train.Saver()
    best_model_path = model_path + '-' + str(epoch)  # 导入最优模型
    saver.restore(sess, best_model_path)
    print('Finished loading model.')
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in tqdm(xrange(n_te_batches)):
        [X1_batch, X2_batch, X3_batch, X4_batch] = get_test_batch(i)
        _batch_size = len(X1_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch,
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    return np.vstack(predict_scores)

# sum_scores = np.zeros((217360, 1999), dtype=float)   
# valid_epoch = [5,6,7]
# for epoch in valid_epoch:
#     score = predict(epoch)
#     sum_scores = sum_scores + score
# print('sum_scores.shape=', sum_scores.shape)
# np.save('../scores/' + model_name + '-all.npy', sum_scores)


# 保存最好的模型
score = predict(7)
print(score.shape)
np.save('../scores/'+model_name + '.npy', score)

local predicting ...
INFO:tensorflow:Restoring parameters from ../ckpt/c1-2-cnn-256-345710/model.ckpt-7


  0%|          | 0/1699 [00:00<?, ?it/s]

Finished loading model.


100%|██████████| 1699/1699 [04:19<00:00,  4.31it/s]


(217360, 1999)


In [11]:
%time predict_labels_list,predict_scores = predict()
df_result = write_result(predict_labels_list, result_path=result_path) 
print('len(df_result)=',len(df_result))  # 结果应该为 217360
print('Saving the predict_scores into %s' % scores_path)
print('predict_scores.shape=',predict_scores.shape)
np.save(scores_path, predict_scores)
print('Finished saving the result!')

CPU times: user 1min 14s, sys: 12.8 s, total: 1min 27s
Wall time: 1min 23s
Finished writing the result
('len(df_result)=', 217360)
Saving the predict_scores into ../scores/p3-2-cnn-256-2357.npy
('predict_scores.shape=', (217360, 1999))
Finished saving the result!


In [12]:
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-7506384235581390893,2919247920214845195,-6306904715218704629,-5932391056759866388
1,6649324930261961840,-240041917918953337,2858911571784840089,3418451812342379591,2382911985227044227,3383016985780045156
2,-4251899610700378615,2919247920214845195,-5265476641576484497,-7270992690764838239,2816249700493135244,-7358589937244777363
3,6213817087034420233,-8655945395761165989,5804619920623030604,-4966205278807386328,-5985158251850250708,7476760589625268543
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,1870872991887862017,-985918886318729368


In [9]:
# 参考结果
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-6839713564940654454,-6306904715218704629,2919247920214845195,-8091907016971478715
1,6649324930261961840,3418451812342379591,2858911571784840089,2382911985227044227,-240041917918953337,3383016985780045156
2,-4251899610700378615,2919247920214845195,-7358589937244777363,2816249700493135244,-5265476641576484497,-3315241959305847628
3,6213817087034420233,-8655945395761165989,5804619920623030604,-4966205278807386328,7476760589625268543,-2523521411748733187
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,1870872991887862017,6018641953300645757


## 在全部预测正确的情况下，理论值为：f1=0.713933
precision=2.50273, recall=0.998873, f1=0.713933

In [26]:
# 假设全部正确，f1 值最高能到多少
def padding_label(labels):
    """把所有的label补齐到长度为 5"""
    label_len = len(labels)
    if label_len >= 5:
        return labels[:5]
    return np.hstack([labels, np.zeros(5-label_len, dtype=int) - 1])
    

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
predict_labels_list = map(padding_label, marked_labels_list)
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print '在全部预测正确的情况下，理论值为：'
print 'precision=%g, recall=%g, f1=%g' % (precision, recall, f1)

在全部预测正确的情况下，理论值为：
precision=2.50273, recall=0.998873, f1=0.713933
